Skrypt poniżej jest składową częścią mojej pracy magisterskiej w której tworzę sieć neuronową, która będzie potrafiła określić czy w danym fragmencie DNA znajduje się funkcjonalny gen kodujący białko u prokarionów i archeonów.

In [2]:
import re
import numpy as np
import pandas as pd
from Bio import Entrez
from Bio import SeqIO, SeqFeature

Funkcje:

hot_encoding - służy do przekonwertowania seq DNA na macierz zrozumiałą dla NN

potnij_seq - funkcja tnie genom pobranego organizmu oraz adnotuje miejsce gdzie znajduje się cięcie

adnotuj_geny - funkcja sprawdza w których fragmentach genomu zachowały się całkowite ramki odczytu genów

In [6]:
def hot_ecoding(sequences):

    dict = {'A': np.array([0.,0.,0.,1.]), 'G': np.array([0.,0.,1.,0.]), 'T': np.array([0.,1.,0.,0.]), 'C': np.array([1.,0.,0.,0.])}

    Seq_list = []

    for sequence in sequences:
        NEW_SEQ = np.empty([4], int)
        
        for nucleotide in sequence:
            NEW_SEQ = np.vstack((NEW_SEQ, dict[nucleotide]))
        
        NEW_SEQ = np.delete(NEW_SEQ, 0, 0)
        Seq_list.append(NEW_SEQ.transpose())
    
   
    return Seq_list

def potnij_seq(sequence):
    
    n = 1000 # Tutaj precyzujemy na jakie fragmenty tniemy
    iteration_start = []
    iteration_end = []
    iteration_counter_start = 1
    iteration_counter_end = n
    
    string = str(sequence)
    
    result = ([string[i:i+n] for i in range(0, len(string), n)])
    for i in result:
        
        iteration_start.append(iteration_counter_start)
        iteration_end.append(iteration_counter_end)
        
        iteration_counter_start += len(i)
        iteration_counter_end += len(i)
    
        
    data = {"Snip":result, "Start":iteration_start, "End":iteration_end}
    df = pd.DataFrame(data)
    
    return df

def adnotuj_geny(df):
    
    is_there_gene = []

    for i in range(0, len(genom)):
        state = False
        for j in range(0, len(gene)):
            if genom['Start'][i] <= int(gene[0][j]) and genom['End'][i] >= int(gene[1][j]):
                is_there_gene.append(1)
                state = True
                break
        if state == False:
            is_there_gene.append(0)
                    
    return is_there_gene

Pobieranie genomów
Wykorzystując moduł Entrez z pakietu Biopythona, bezpośrednio pobieram recordy genomów w formacie genbank posiadających wszystkie charakterystyki opisowe genomu.

W przyszłości skrypt poniżej zostanie zmodyfikowany, będzie można pobierać wiele genomów jednocześnie, określonych w oddzielnym pliku, oraz równocześnie transformować dane do porządanego formatu, który akceptuje NN.

In [3]:
Entrez.email = "dawid.gazda99@gamil.com"
handle = Entrez.esearch(db="nucleotide", term="NC_000913")
record = Entrez.read(handle)
record["Count"]

'1'

In [ ]:
handle = Entrez.efetch(db="nucleotide", id=record['IdList'], rettype="gbwithparts", retmode="text")
Entrez.email = "dawid.gazda99@gmail.com"
record = SeqIO.read(handle, "gb")

outputname="/Users/dawidgazda/Desktop/Projekty/Python/genome.gb"

In [ ]:
SeqIO.write(record, outputname, 'gb')

Tutaj zachodzi ekstrakcja, z pliku z genomem, adnotowanych miejsc na których znajdują się geny.

In [4]:
annot = []

with open('/Users/dawidgazda/Desktop/Projekty/Python/genome.gb') as handle:
    for record in SeqIO.parse(handle, 'gb'):
        for feature in record.features:
            if feature.type =='gene' and 'CDS':
                annot.append(str(feature.location))

annot_1 = []
for string in annot:
    nowy_string = re.sub(r'\(.*\)','',string)
    annot_1.append(nowy_string[1:-1])
    
gene = pd.Series(annot_1)
gene = gene.str.split(':', expand=True)

Poniżej używam zdefiniowanych wcześniej funkcji do transformacji pobranego wcześniej genomu

Docelowo chcę otrzymać plik zawierający zakodowane fragmenty DNA, w formie macierzy, wraz z adnotacją czy znajduje się w nich funkcjonalny gen.

In [7]:
with open('/Users/dawidgazda/Desktop/Projekty/Python/genome.gb') as handle:
    for record in SeqIO.parse(handle, 'gb'):
        genom = potnij_seq(str(record.seq))

CDS = adnotuj_geny(genom)
Snip_encoded = hot_ecoding(genom['Snip'])

In [8]:
koncowy_wynik = pd.DataFrame()
koncowy_wynik['Snip'] = Snip_encoded
koncowy_wynik['Gene'] = CDS
koncowy_wynik.to_csv('train_data.csv', index=False)